In [77]:
import pandas as pd
import numpy as np
WDIR = './yelp_dataset/'

In [78]:
business_json_path = WDIR + "yelp_academic_dataset_business.json"
checkin_json_path = WDIR + "yelp_academic_dataset_checkin.json"
tip_json_path = WDIR + "yelp_academic_dataset_tip.json"
reviews_json_path = WDIR + "yelp_academic_dataset_review.json"

In [79]:
business_json_path

'./yelp_dataset/yelp_academic_dataset_business.json'

In [80]:
# Read in businesses df
df_b = pd.read_json(business_json_path, lines=True)

In [4]:
# Maintain a relevant set of businesses
relevant_businesses = set(df_b[df_b['state'].isin(['VA', 'NC'])].business_id.unique())

# Utility function to read in large dataframes and filter on the fly
def filter(path, critera, size=1000):
    chunk_list = []
    reader = pd.read_json(path, lines=True, chunksize=size)
    
    for chunk in reader:
        chunk_filter = chunk[chunk.business_id.isin(critera)]
        chunk_list.append(chunk_filter)
    
    reader.close()
    return pd.concat(chunk_list)

In [ ]:
# Read and save reviews file
df_revs = filter(reviews_json_path, relevant_businesses, 1000)
df_concat.to_pickle(WDIR + 'reviews.pkl')

In [ ]:
# Read and save checkin file
df_checkin = filter(checkin_json_path, relevant_businesses, 1000)
df_checkin.to_pickle(WDIR + 'checkin.pkl')

In [ ]:
# Read and save tip file
df_tips = filter(tip_json_path, relevant_businesses, 1000)
df_tips.to_pickle(WDIR + 'tips.pkl')

In [5]:
import pickle
with open('reviews.pkl', 'rb') as pickle_file:
    df_revs = pickle.load(pickle_file)
    
with open('checkin.pkl', 'rb') as pickle_file:
    df_checkin = pickle.load(pickle_file)
    
with open('tips.pkl', 'rb') as pickle_file:
    df_tips = pickle.load(pickle_file)

In [19]:
df_checkin.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
29,oyjkEn5fAqcGNsBbqc8LTw,0,2016-04-11 16:42:13,0,01DNCgnW8K6xSyv4krYa1w,1,Just... not good. We want it to be - we have ...,0,XivuRnPq3D-K5cPiBTztIA
58,l4ByLNdIcsjgnglB1aXQQQ,0,2018-04-15 14:54:30,0,BnB8z3d1BiLyqdAwgQnejw,1,I can't tell you how angry I am right now. I ...,1,fn0wmD6MY2zmNUkOJJrjjA
64,UOFTGqJ95xQxeYndG7hbbw,1,2011-08-25 00:49:25,0,PvgCQa3FsA-FXYQsaEguIw,5,I was staying at the Hilton next door and got ...,0,LhZhxlOqTSuRY6r7HV6NVg
66,gfhWUixZ4MmEAC1DtTsodA,0,2017-06-06 21:04:58,0,2dYcxwdtulsUY8kj4oZezw,5,GREAT SERVICE!! The food was amazing and very ...,0,iAQFEArbXp1WUrseOFTLMA
78,l_kefVF1frmC0xRW2YkvUA,0,2018-01-21 04:58:46,0,MRPCm_BYEHl2T9oNZYnCbQ,2,"Pros: Fun atmosphere, great for people watchin...",2,oAFF_A02Ws0mwH-lkbzyMQ


In [7]:
df_tips.head()

,business_id,compliment_count,date,text,user_id
2,rDoT-MgxGRiYqCmi0bG10g,0,2016-07-18 22:03:42,Very nice good service good food,AY-laIws3S7YXNl_f_D6rQ
5,ALwAlxItASeEs2vYAeLXHA,0,2015-05-20 20:17:38,Great ramen! Not only is the presentation gorg...,HHNBqfbDR8b1iq-QGxu8ww
9,hfBrethLHS9iXeBNR8MtzQ,0,2011-09-30 18:38:47,Check out the great assortment of organic & co...,DsWg3leomfasGs3j0rOfbQ
12,gG9z6zr_49LocyCTvSFg0w,0,2014-06-14 00:44:00,Just be ready to stand in line. But it moves a...,oLHQ-31LcRobRxjRU6PrZA
25,a4grXFNKZmZm0ZJXd1PVdQ,0,2012-01-01 02:33:27,Great views of the city!,exEElE4pG-ZwoCMwoYddeg


In [14]:
df_revs.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
29,oyjkEn5fAqcGNsBbqc8LTw,0,2016-04-11 16:42:13,0,01DNCgnW8K6xSyv4krYa1w,1,Just... not good. We want it to be - we have ...,0,XivuRnPq3D-K5cPiBTztIA
58,l4ByLNdIcsjgnglB1aXQQQ,0,2018-04-15 14:54:30,0,BnB8z3d1BiLyqdAwgQnejw,1,I can't tell you how angry I am right now. I ...,1,fn0wmD6MY2zmNUkOJJrjjA
64,UOFTGqJ95xQxeYndG7hbbw,1,2011-08-25 00:49:25,0,PvgCQa3FsA-FXYQsaEguIw,5,I was staying at the Hilton next door and got ...,0,LhZhxlOqTSuRY6r7HV6NVg
66,gfhWUixZ4MmEAC1DtTsodA,0,2017-06-06 21:04:58,0,2dYcxwdtulsUY8kj4oZezw,5,GREAT SERVICE!! The food was amazing and very ...,0,iAQFEArbXp1WUrseOFTLMA
78,l_kefVF1frmC0xRW2YkvUA,0,2018-01-21 04:58:46,0,MRPCm_BYEHl2T9oNZYnCbQ,2,"Pros: Fun atmosphere, great for people watchin...",2,oAFF_A02Ws0mwH-lkbzyMQ


In [9]:
df_b.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,10913 Bailey Rd,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",f9NumwFMBDn751xgFiRbNA,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",Cornelius,"{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",1,35.462724,-80.852612,The Range At Lake Norman,28031,36,3.5,NC
1,"8880 E Via Linda, Ste 107","{'GoodForKids': 'True', 'ByAppointmentOnly': '...",Yzvjg0SayhoZgCljUJRF9Q,"Health & Medical, Fitness & Instruction, Yoga,...",Scottsdale,None,1,33.569404,-111.890264,"Carlos Santo, NMD",85258,4,5.0,AZ
2,3554 Rue Notre-Dame O,None,XNoUzKckATkOD1hP6vghZg,"Pets, Pet Services, Pet Groomers",Montreal,None,1,45.479984,-73.580070,Felinus,H4C 1P4,5,5.0,QC
3,1015 Sharp Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",6OAZjbxqM5ol29BuHsil3w,"Hardware Stores, Home Services, Building Suppl...",North Las Vegas,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",0,36.219728,-115.127725,Nevada House of Hose,89030,3,2.5,NV
4,4827 E Downing Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",51M2Kk903DFYI6gnB5I6SQ,"Home Services, Plumbing, Electricians, Handyma...",Mesa,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",1,33.428065,-111.726648,USE MY GUY SERVICES LLC,85205,26,4.5,AZ


In [10]:
df_b[df_b['is_open'] == 0]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
3,1015 Sharp Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",6OAZjbxqM5ol29BuHsil3w,"Hardware Stores, Home Services, Building Suppl...",North Las Vegas,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",0,36.219728,-115.127725,Nevada House of Hose,89030,3,2.5,NV
14,5465 Simmons St,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",kOICO53wbOiOJcKuCgOQ3A,"Beauty & Spas, Tanning",North Las Vegas,"{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...",0,36.263362,-115.179839,Tan Las Vegas,89031,5,3.5,NV
15,2402 Atchley Dr,{'BusinessAcceptsCreditCards': 'True'},rqcOZePlVvJP9EtzldIz0w,"Local Services, Self Storage",Henderson,"{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",0,35.952046,-115.093483,Desert Storage,89052,5,5.0,NV
16,3131 Las Vegas Blvd,"{'BusinessParking': '{'garage': True, 'street'...",uZuh51lXu7tsrC8RAwkg1A,"Shopping, Shoe Stores, Fashion",Las Vegas,None,0,36.125732,-115.167608,Manolo Blahnik,89109,32,4.5,NV
20,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,None,0,35.194894,-80.767442,Middle East Deli,28205,5,3.0,NC
29,415 Horner Avenue,"{'RestaurantsPriceRange2': '3', 'BikeParking':...",007Dg4ESDVacWcC4Vq704Q,"Shopping, Food, Organic Stores, Specialty Food...",Toronto,"{'Monday': '9:0-16:0', 'Tuesday': '9:0-16:0', ...",0,43.603232,-79.538424,Front Door Organics,M8W 4W3,8,4.0,ON
32,1255 W Main St,"{'BikeParking': 'True', 'BusinessParking': '{'...",DCsS3SgVFO56F6wRO_ewgA,"Donuts, Juice Bars & Smoothies, Food, Coffee &...",Mesa,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",0,33.414409,-111.858378,Missy Donuts & Coffee,85201,7,2.5,AZ
36,9595 W Tropicana Ave,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...",fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food",Las Vegas,None,0,36.099738,-115.301568,Carl's Jr,89147,15,2.5,NV
38,843 Kipling Avenue,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...",rVBPQdeayMYht4Uv_FOLHg,"Restaurants, Burgers, Food",Toronto,None,0,43.633291,-79.531768,Gourmet Burger Company,M8Z 5G9,13,3.0,ON
42,6530 S Decatur Blvd,"{'RestaurantsPriceRange2': '2', 'BusinessParki...",fhNf_sg-XzZ3e7HEVGuOZg,"Burgers, Restaurants, American (Traditional), ...",Las Vegas,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0,36.071196,-115.207463,Meat Chix And Wieners,89118,10,3.0,NV


In [22]:
bus_rev_dict = {}
for x in relevant_businesses:
    bus_rev_dict[x] = []
for x,c in df_revs.iterrows():
    try:
        bus_rev_dict[c['business_id']].append(c['text'])
    except:
        pass

In [23]:
for x,c in df_revs.iterrows():
    try:
        bus_rev_dict[c['business_id']].append(c['text'])
    except:
        pass

In [44]:
from textblob import TextBlob
import nltk

In [45]:
def findavgsentiment(textarr):
    totalSent = 0
    for text in textarr:
        blob = TextBlob(text)
        totalSent += blob.sentiment.polarity
    return totalSent/len(textarr)

In [52]:
bus_score_dict = {}
for x in relevant_businesses:
    bus_score_dict[x] = []
for item in bus_rev_dict.items():
    bus_score_dict[item[0]] = findavgsentiment(item[1])
    

In [53]:
bus_score_dict

{'fJkXfvCiVGJsjrIrUuLJCQ': 0.26234487508453674,
 'FqVTaF56MSgqCCX3N6hgLQ': 0.37071906565656565,
 'sV6kKpzneWwQ8cY8gQ8bgA': 0.263858991033446,
 'R2ir667KUOdyhGJp9Q06dQ': 0.24828639331498806,
 '4GrFgFdt3fYdZsYbnyRlZg': 0.10212637741046832,
 'x5fgrhaq8R8qXhZFPnfXWw': 0.2849578781257856,
 'VKdVPVkMOotZ-J248lHReQ': 0.25297619047619047,
 'pCb0jYxksLR562vV7As1bQ': 0.2867355784277791,
 'GlQs6zeCN0dwCe7gMBXFsg': 0.3225956664862915,
 'jdft5jzXyceJZLRtVL38VA': 0.13932570912903125,
 'iqx8VckOhO7nict32SPjKg': 0.13101826306357292,
 'wUHPgIh-rJThxPvxHPlsjQ': 0.1637833522721307,
 '4y8LLc9tuxqKhDSeRDUTDA': 0.2587474829916635,
 'huTkwb90PQw4HqGkbhz74Q': 0.2993022486772487,
 'VXFycyyI3uEx6IOHkPLvSQ': 0.3499585578832374,
 'vi2UlcBHPlGMTbGWO1e2-w': 0.3531944444444444,
 'l9tFVg7r7QXuLnCywMQl7Q': 0.29586083920224054,
 'ic5TJB0pUeiQ53J2CRBMig': 0.15404577234934375,
 'esB5D7EbZ5mJkqwSBK1yOA': -0.00465277777777779,
 '1Dx2W-2GL9LaE9j2vD5lfw': 0.13834928075637923,
 'at63ej-ri33Vqx4333Rd3Q': 0.12056925047861677,
 